# Specimen testing

`modkit pileup kasumi1_naive.bam kasumi1_pileup_cpg.bed --combine-mods --log-filepath pileup.log --no-filtering -t 32 --combine-strands --cpg --ref GCA_000001405.15_GRCh38_no_alt_analysis_set.fna`

## Where data at?

In [4]:
import pandas as pd

mount = '/mnt/e/'
# input_path = mount + 'MethylScore/Nanopore_data/Kasumi1-naive-p2solo.bed.gz'
# input_path = '/mnt/h/UF_HemBank_1852/SAMPLE_ungrouped.wf_mods.bedmethyl.gz'
# input_path = '/mnt/h/epi2me_analyses/uf_hembank_1832_wf-human-variation_01HQG462AJKW6RH7XCFBQPD54E/output/UF_HemBank_1832_ungrouped.wf_mods.bedmethyl.gz'
input_path = '~/projects/modkit_runs/kasumi1_pileup_cpg_pacmap.bed'
reference_path = mount + 'genome_references/Illumina_methylation_arrays/EPIC.anno.GRCh38.tsv.gz'
output_path = mount + 'MethylScore_v2/Processed_Data/'

sample_name = 'kasumi1_naive'

## Harmonize probes from EPIC array with nanopore methylation calls

In [5]:
# read df_discovery and df_validation
# df_discovery = pd.read_pickle(mount+'MethylScore/Intermediate_Files/'+'3308samples_333059cpgs_withbatchcorrection_bvalues.pkl').sort_index().iloc[:,1:]
# array_reference = pd.read_csv(reference_path, sep='\t', usecols=['chrm','start','end','probeID']).set_index('probeID')
# array_reference['coordinate'] = array_reference['chrm'].astype(str) + ':' + array_reference['start'].astype(str)
# pacmap_reference = array_reference.loc[df_discovery.columns]
# pacmap_reference.to_pickle(mount+'MethylScore/Intermediate_Files/'+'pacmap_probe_reference.pkl')

# load the reference data
pacmap_reference = pd.read_pickle(mount+'MethylScore_v2/Intermediate_Files/'+'pacmap_probe_reference.pkl').reset_index().set_index('coordinate')

# Save `pacmap_reference` as a .bed file
# pacmap_reference[['chrm', 'start', 'end',]].to_csv('~/projects/modkit_runs/pacmap_reference.bed', sep='\t', header=False, index=False)

# Define columns to be used for the input data
usecols = [0, 1, 4, 10]
column_names = ["chrom", "start_position", "score", "fraction_modified"]

# Read the input data, skipping the first row if it's a header or irrelevant
df = pd.read_csv(input_path, sep='\s+', skiprows=1, names=column_names, usecols=usecols)

# Create 'coordinate' column for merging
df['coordinate'] = df['chrom'].astype(str) + ':' + df['start_position'].astype(str)

In [11]:
df_filtered = df.set_index('coordinate')

# Join with reference data on 'coordinate'
df_merged = df_filtered.join(pacmap_reference, how='outer')

In [13]:

# # Interpolate missing values in 'fraction_modified' column linearly
df_merged['fraction_modified'] = df_merged['fraction_modified'].astype(float).interpolate(method='linear')

df_merged = df_merged[['fraction_modified']].join(pacmap_reference, how='inner')

df_merged = df_merged.drop_duplicates(subset='IlmnID')

# Calculate the fraction_modified and prepare the final DataFrame
df_merged.loc[:, sample_name] = (df_merged['fraction_modified'] / 100).round(3)

df_processed = df_merged[['IlmnID', sample_name]].set_index('IlmnID').T

In [3]:
# load the reference data
pacmap_reference = pd.read_pickle(mount+'MethylScore/Intermediate_Files/'+'pacmap_probe_reference.pkl').reset_index().set_index('coordinate')

# Define columns to be used for the input data
usecols = [0, 1, 3, 10]
column_names = ["chrom", "start_position", "modified_base_code", "fraction_modified"]

# Read the input data, skipping the first row if it's a header or irrelevant
df = pd.read_csv(input_path, sep='\s+', skiprows=1, names=column_names, usecols=usecols)

# Create 'coordinate' column for merging
df['coordinate'] = df['chrom'].astype(str) + ':' + df['start_position'].astype(str)

# Filter for 'm' and 'h' modified base codes before merging
df_filtered = df[df['modified_base_code'].isin(['m'])].set_index('coordinate')

In [4]:
array_reference = pd.read_csv(reference_path, sep='\t', usecols=['chrm','start','end','probeID']).set_index('probeID')
array_reference['coordinate'] = array_reference['chrm'].astype(str) + ':' + array_reference['start'].astype(str)

In [5]:
# Merge the two dataframes
df_5mc_merged = pd.merge(df_filtered, array_reference.reset_index()[['probeID','coordinate']], on='coordinate', how='inner')

# Set the index to the probeID
df_5mc_merged = df_5mc_merged.set_index('probeID')

# Create beta values column
df_5mc_merged[sample_name] = (df_5mc_merged['fraction_modified'] / 100).round(3)

# Create a new dataframe with only the beta values
df_nanopore = df_5mc_merged[[sample_name]].T

df_nanopore

probeID,cg14817997,cg26928153,cg16269199,cg13869341,cg14008030,cg18231760,cg20253340,cg02404219,cg04098293,cg24335620,...,cg00491786,cg07587934,cg16855331,cg05740793,cg20450977,cg11894324,cg24159721,cg05001044,cg08858441,cg03348902
UF_HemBank_1832,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.333,0.0,0.0,0.0,0.0,0.0,0.001,0.0,0.001


In [6]:
df_discovery = pd.read_pickle(mount+'MethylScore/Intermediate_Files/'+'3308samples_333059cpgs_withbatchcorrection_bvalues.pkl').sort_index().iloc[:,1:]

In [7]:
# use overlapping features between df_discovery and df_nanopore
common_features = [x for x in df_discovery.columns if x in df_nanopore.columns]

# apply `common_features` to both df_discovery and df_nanopore
df_discovery = df_discovery[common_features]
df_nanopore = df_nanopore[common_features]

In [8]:
import pacmap
print('PaCMAP version:', pacmap.__version__)

components_list = [5]
for components in components_list:
    
    reducer = pacmap.PaCMAP(n_components=components, 
                            n_neighbors=10, 
                            MN_ratio=0.4, 
                            FP_ratio=16.0, 
                            lr=0.1, 
                            num_iters=5000,
                            random_state=42,
                            save_tree=True)

    # Project the high dimensional dataset into a low-dimensional embedding
    embedding_training = reducer.fit_transform(df_discovery.to_numpy(dtype='float16'))

    # Save reducer
    # pacmap.save(reducer, f'../models/pacmap_{components}d_model_al_atlas')

    # Create column names
    cols = ['PaCMAP '+ str(i+1) + f' of {components}' for i in range(components)]

    # Turn embedding into dataframe
    df_embedding = pd.DataFrame(embedding_training, columns=cols, index=df_discovery.index)

PaCMAP version: 0.7.0


/home/fmarchi/projects/AML-atlas-notebooks/.venv/lib/python3.8/site-packages/pacmap/pacmap.py:819: UserWarning: Warning: random state is set to 42
  warnings.warn(f'Warning: random state is set to {_RANDOM_STATE}')


## Apply PaCMAP model

In [10]:
import pacmap

def apply_pacmap_model_to_new_data(df, components, reducer=reducer):

    # Project the high dimensional dataset into existing embedding space and return the embedding.
    embedding = reducer.transform(df.to_numpy(dtype='float16'))

    # Create column names
    cols = ['PaCMAP '+ str(i+1) + f' of {components}' for i in range(components)]

    # Turn embedding into dataframe
    df_embedding = pd.DataFrame(embedding, columns=cols, index=df.index)

    return df_embedding


df_embedding_5d = apply_pacmap_model_to_new_data(df_nanopore, 5)

In [11]:
df_embedding_5d

,PaCMAP 1 of 5,PaCMAP 2 of 5,PaCMAP 3 of 5,PaCMAP 4 of 5,PaCMAP 5 of 5
UF_HemBank_1832,-13.868196,-15.072299,13.773742,-22.246822,10.254135


In [ ]:
# import pacmap

# def apply_pacmap_model_to_new_data(df, components):

#     # Load reducer
#     reducer = pacmap.load(f'../models/pacmap_{components}d_model_al_atlas')

#     # Project the high dimensional dataset into existing embedding space and return the embedding.
#     embedding = reducer.transform(df.to_numpy(dtype='float16'))

#     # Create column names
#     cols = ['PaCMAP '+ str(i+1) + f' of {components}' for i in range(components)]

#     # Turn embedding into dataframe
#     df_embedding = pd.DataFrame(embedding, columns=cols, index=df.index)

#     return df_embedding

# df_embedding_2d = apply_pacmap_model_to_new_data(df_processed, 2)
# df_embedding_5d = apply_pacmap_model_to_new_data(df_processed, 5)

# df_embedding_2d.to_pickle(output_path + sample_name + '_pacmap_2d.pkl')
# df_embedding_5d.to_pickle(output_path + sample_name + '_pacmap_5d.pkl')

## Apply supervised models

In [56]:
import joblib

# Load models
lgbm_px_model = joblib.load('../models/lgbm_px_model.pkl')
lgbm_dx_model = joblib.load('../models/lgbm_dx_model.pkl')

# load `df_embedding_5d` from the previous step
df_embedding_5d = pd.read_pickle(output_path + sample_name + '_pacmap_5d.pkl')

def save_predictions(df, classifier, model_name):

    # ignore sklearn warnings
    import warnings
    warnings.filterwarnings('ignore')

    # Select necessary columns
    df_features = df.copy()

    # Predict using the selected columns
    predictions = classifier.predict(df_features)

    # Predict probabilities using the selected columns
    probabilities = classifier.predict_proba(df_features)

    # Convert predictions to a Series with the same index as df_features
    predictions_series = pd.Series(predictions, index=df_features.index, name=model_name)

    # Convert probabilities to a DataFrame with the same index as df_features and the same columns as the classes
    probabilities_df = pd.DataFrame(probabilities, index=df_features.index, columns=classifier.classes_).round(3)

    # Add " - predict_proba" to the column names
    probabilities_df.columns ='P(' + probabilities_df.columns + ')'

    # Transform classes of the predictions into integers based on unique values in the classes
    probabilities_df[model_name + '_int'] = predictions_series.map({c: i for i, c in enumerate(classifier.classes_)})

    # Join predictions with the original DataFrame (already indexed)
    df_joined = predictions_series.to_frame().join(probabilities_df)

    return df_joined

# Execution
df_pred_px = save_predictions(df=df_embedding_5d, classifier=lgbm_px_model, model_name='AML Epigenomic Risk')
df_pred_dx = save_predictions(df=df_embedding_5d, classifier=lgbm_dx_model, model_name='AL Epigenomic Phenotype')

# Map the classes to more desirable labels (low and high risk)
df_pred_px['AML Epigenomic Risk'] = df_pred_px['AML Epigenomic Risk'].map({'Alive': 'Low', 'Dead': 'High'})
df_pred_px = df_pred_px.rename(columns={'P(Alive)': 'AML Epigenomic Risk P(Low Risk)', 'P(Dead)': 'AML Epigenomic Risk P(High Risk)'})

# Join predictions with clinical data
df_combined = df_embedding_2d.join(df_embedding_5d).join(df_pred_px).join(df_pred_dx)

df_combined[['AML Epigenomic Risk', 'AML Epigenomic Risk P(High Risk)', 'AL Epigenomic Phenotype', f'P({df_combined["AL Epigenomic Phenotype"].item()})']]

,AML Epigenomic Risk,AML Epigenomic Risk P(High Risk),AL Epigenomic Phenotype,P(T-ALL NOS)
UF_HemBank_1832,High,0.546,T-ALL NOS,0.969


## EWASCox-Lasso

In [57]:
import math
import sys
sys.path.append('../')
from source.cox_lasso import *

raw_coefs = pd.read_csv(output_path + 'multivariate_cox_lasso/ewas_cog_os_raw_coefs_newrisk.csv', index_col=0)

mean_coefs = set_cutoff(coefs=raw_coefs,threshold=0.99)

df_validation = df_processed[mean_coefs.index]

df_validation_transformed = df_validation.replace(1, 0.999).replace(0, 0.001)

def beta2m(val):
    '''Transfrom beta-values into m-values'''
    return math.log2(val/(1-val))

x_test_m = df_validation_transformed.apply(np.vectorize(beta2m))

def standardize_data(df, reference_df):
    """Standardize data using mean and standard deviation of reference dataset"""

    # Keep only columns that are in both datasets
    reference_df = reference_df.loc[:, df.columns]

    # Standardize data
    df_z = (df - reference_df.mean()) / reference_df.std()

    return df_z

# Read top CpGs selected from previous code file (univariate cox-ph EWAS)
ewas_top_cpgs = pd.read_csv(output_path+'ewas_dmr/ewas_top_cpgs_os.csv', index_col=0)

# Standardize data
x_test_m_z = standardize_data(df= x_test_m, reference_df= ewas_top_cpgs)

score_name = 'EWASCox_OS_48CpGs'

df_test, threshold = generate_coxph_score(coef_mean=mean_coefs,
                                        x=x_test_m_z,
                                        df=df_validation_transformed,
                                        score_name=score_name,
                                        train_test=0.4934,
                                        rpart_outcome='os.time')

df_validation_transformed[['EWASCox_OS_48CpGs','EWASCox_OS_48CpGs Categorical']]

Continuous score cut at the value of 0.4934


IlmnID,EWASCox_OS_48CpGs,EWASCox_OS_48CpGs Categorical
UF_HemBank_1832,3.236585,High


## Save results

In [58]:
df_nanopore = df_combined.join(df_validation_transformed[['EWASCox_OS_48CpGs','EWASCox_OS_48CpGs Categorical']])

df_nanopore['Train-Test'] = 'Long-read Nanopore sequencing'
df_nanopore['Clinical Trial'] = 'UF Hem Bank'
df_nanopore['Patient_ID'] = sample_name
df_nanopore['Hematopoietic Entity'] = np.nan
df_nanopore['WHO 2022 Diagnosis'] =  np.nan
df_nanopore['Vital Status'] = np.nan
df_nanopore['Risk Group AAML1831'] = np.nan

df_nanopore.to_excel(output_path + sample_name + '_processed.xlsx')

## Watermark

In [59]:
%load_ext watermark

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


In [60]:
# watermark with all libraries used in this notebook
%watermark -v -p numpy,pandas,pacmap,sklearn,lightgbm -a Francisco_Marchi@Lamba_Lab_UF -d -m

Author: Francisco_Marchi@Lamba_Lab_UF

Python implementation: CPython
Python version       : 3.8.16
IPython version      : 8.12.3

numpy   : 1.24.4
pandas  : 2.0.3
pacmap  : 0.7.0
sklearn : 1.2.2
lightgbm: 3.3.5

Compiler    : GCC 11.3.0
OS          : Linux
Release     : 5.15.133.1-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit



```{note}
Please only use the following versions:
`python`: 3.8.16
`pacmap`: 0.7.0
`lightgbm`: 3.3.5
`scikit-learn`: 1.2.2
```